In [40]:
from os import listdir
from os.path import isfile, join
import math
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean

In [41]:
import numpy as np
import scipy.stats


def confidence_interval(data, confidence=0.99):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

In [42]:
root_dir = './results_backup/three_results/'
root_dir_pic = './results_pic/'

results=[]
for f in listdir(root_dir):
    path = join(root_dir, f)
    if isfile(path):
        fo = open(path)
        jso = json.loads(fo.read())
        splt = f.split('_')
        tmpvm = splt[3] if len(splt) == 5 else splt[3] + '_' + splt[4]
        if tmpvm.startswith('openj9'):
                vm = 'openj9'
        elif tmpvm.startswith('hotspot'):
                vm = 'hotspot'
        elif tmpvm.startswith('graalvm-ce'):
                vm = 'graalvm CE'
        else:
                vm = 'graalvm EE'
        if '11' in tmpvm:
                vm_version = 11
        elif '12' in tmpvm:
                vm_version = 12
        else:
                vm_version = 8
        timestamp = splt[-1][:-5]
        pattern = splt[1]
        program = splt[2]
        language = splt[0]
        for (idx, time) in enumerate(jso[0]['primaryMetric']['rawData'][0]):
                results.append({
                        'vm': vm,
                        'vm_version': str(vm_version),
                        'vm_full': vm + ' ' + str(vm_version),
                        'pattern': pattern,
                        'program': str(language[0]+"_"+program),
                        'language': language,
                        'timestamp': timestamp,
                        'time': time,
                        'idx': idx
                })

for f in listdir(root_dir_pic):
    path = join(root_dir_pic, f)
    if isfile(path):
        fo = open(path)
        jso = json.loads(fo.read())
        splt = f.split('_')
        tmpvm = splt[3] if len(splt) == 5 else splt[3] + '_' + splt[4]
        if tmpvm.startswith('openj9'):
                vm = 'openj9'
        elif tmpvm.startswith('hotspot'):
                vm = 'hotspot'
        elif tmpvm.startswith('graalvm-ce'):
                vm = 'graalvm CE'
        else:
                vm = 'graalvm EE'
        if '11' in tmpvm:
                vm_version = 11
        elif '12' in tmpvm:
                vm_version = 12
        else:
                vm_version = 8
        timestamp = splt[-1][:-5]
        program = splt[2]
        language = splt[0]
        for (idx, time) in enumerate(jso[0]['primaryMetric']['rawData'][0]):
                
                results.append({
                        'vm': vm,
                        'vm_version': str(vm_version),
                        'vm_full': vm + ' ' + str(vm_version),
                        'pattern': 'truffle + pic',
                        'program': str(language[0]+"_"+program),
                        'language': language,
                        'timestamp': timestamp,
                        'time': time,
                        'idx': idx
                })

#results = sorted(results, key=lambda x: x['program'])
df = pd.DataFrame(results).sort_values(by=['program'])
for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
    print(c, df[c].unique())
df.describe(include='all')

vm ['openj9' 'hotspot' 'graalvm CE' 'graalvm EE']
vm_version ['12' '8' '11']
vm_full ['openj9 12' 'hotspot 8' 'graalvm CE 8' 'hotspot 12' 'openj9 8'
 'graalvm EE 8' 'hotspot 11' 'openj9 11']
pattern ['visitor' 'revisitor' 'interpreter' 'switch' 'truffle' 'truffle + pic']
program ['b_fibonacci' 'b_sort' 'f_buffers' 'l_fractal' 'm_binarytree'
 'm_fannkuchredux' 'm_fibonacci' 'm_sort']
language ['boa' 'fsm' 'logo' 'minijava']


,vm,vm_version,vm_full,pattern,program,language,timestamp,time,idx
count,42450,42450,42450,42450,42450,42450,42450,42450.000000,42450.00000
unique,4,3,8,6,8,4,849,NaN,NaN
top,hotspot,8,graalvm EE 8,revisitor,m_fibonacci,minijava,1564897640502,NaN,NaN
freq,14400,23250,6850,9600,5400,21600,50,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.293558,24.50000
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.981974,14.43104
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.817689,0.00000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.984415,12.00000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.275266,24.50000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.613677,37.00000


In [43]:
print('Hotspot VM (Switch)')
hotspot_switch={}
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "hotspot" & program == "%s" & pattern == "switch" & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    hotspot_switch[p] = { 'mean': mean, 'ci': confidence_interval(times)}
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
pd.DataFrame(hotspot_switch)

Hotspot VM (Switch)
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['b_fibonacci'],language:['boa'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['b_sort'],language:['boa'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['f_buffers'],language:['fsm'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['l_fractal'],language:['logo'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_binarytree'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_fannkuchredux'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_fibonacci'],language:['minijava'],
vm:['hotspot'],vm_version:['8'],vm_full:['hotspot 8'],pattern:['switch'],program:['m_sort'],language:['minijava'],


,b_fibonacci,b_sort,f_buffers,l_fractal,m_binarytree,m_fannkuchredux,m_fibonacci,m_sort
mean,2.044608,1.710339,7.837011,12.440172,7.057711,6.666758,11.797858,16.391664
ci,0.039194,0.013048,0.034134,0.059237,0.034380,0.029380,0.017202,0.017569


In [44]:
print('Hotspot VM (Interpreter)')
hotspot_interpreter={}
mean_shs=[]
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "hotspot" & program == "%s" & pattern == "interpreter" & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    shs=hotspot_switch[p]['mean'] / mean
    mean_shs.append(shs)
    hotspot_interpreter[p] = { 'mean': mean, 'ci': confidence_interval(times), 'shs': shs}
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
print('mean shs', gmean(mean_shs))
pd.DataFrame(hotspot_interpreter)

Hotspot VM (Interpreter)
mean shs 1.2127438904450674


,b_fibonacci,b_sort,f_buffers,l_fractal,m_binarytree,m_fannkuchredux,m_fibonacci,m_sort
mean,1.792009,1.940170,4.044563,11.141054,5.858538,5.276449,9.181957,14.909981
ci,0.063757,0.012230,0.017250,0.054291,0.009663,0.020407,0.016980,0.035728
shs,1.140959,0.881541,1.937666,1.116606,1.204688,1.263493,1.284896,1.099375


In [45]:
print('Graal VM (Interpreter)')
graal_interpreter={}
means_shs=[]
means_shi=[]
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "graalvm CE" & program == "%s" & pattern == "interpreter"  & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    means_shs.append(hotspot_switch[p]['mean'] / mean)
    means_shi.append(hotspot_interpreter[p]['mean'] / mean)
    graal_interpreter[p] = {
        'mean': mean,
        'ci': confidence_interval(times),
        'shi': hotspot_interpreter[p]['mean'] / mean,
        'shs': hotspot_switch[p]['mean'] / mean,
    }
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
print('mean shi', gmean(means_shi))
print('mean shs', gmean(means_shs))
pd.DataFrame(graal_interpreter)

Graal VM (Interpreter)
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['b_fibonacci'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['b_sort'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['f_buffers'],language:['fsm'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['l_fractal'],language:['logo'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['m_binarytree'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['m_fannkuchredux'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter'],program:['m_fibonacci'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['interpreter']

,b_fibonacci,b_sort,f_buffers,l_fractal,m_binarytree,m_fannkuchredux,m_fibonacci,m_sort
mean,1.465116,1.571188,4.636862,7.869716,3.316034,3.174092,5.292045,9.890358
ci,0.024003,0.001991,0.009169,0.041039,0.004565,0.012101,0.006172,0.009744
shi,1.223117,1.234843,0.872263,1.415687,1.766730,1.662349,1.735049,1.507527
shs,1.395526,1.088564,1.690154,1.580765,2.128359,2.100367,2.229357,1.657338


In [46]:
print('Graal VM (Interpreter)')
truffle_interpreter={}
means_shs=[]
means_shi=[]
means_sgi=[]
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "graalvm CE" & program == "%s" & pattern == "truffle"  & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    means_shs.append(hotspot_switch[p]['mean'] / mean)
    means_shi.append(hotspot_interpreter[p]['mean'] / mean)
    means_sgi.append(graal_interpreter[p]['mean'] / mean)
    truffle_interpreter[p] = {
        'mean': mean,
        'ci': confidence_interval(times),
        'sgi': graal_interpreter[p]['mean'] / mean,
        'shi': hotspot_interpreter[p]['mean'] / mean,
        'shs': hotspot_switch[p]['mean'] / mean,
    }
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
print('mean sgi', gmean(means_sgi))
print('mean shi', gmean(means_shi))
print('mean shs', gmean(means_shs))
pd.DataFrame(truffle_interpreter)

Graal VM (Interpreter)
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['b_fibonacci'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['b_sort'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['f_buffers'],language:['fsm'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['l_fractal'],language:['logo'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['m_binarytree'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['m_fannkuchredux'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['m_fibonacci'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle'],program:['m_sort'],language:['m

,b_fibonacci,b_sort,f_buffers,l_fractal,m_binarytree,m_fannkuchredux,m_fibonacci,m_sort
mean,1.391093,1.593299,3.937675,8.048241,3.311739,3.027044,4.673049,10.258824
ci,0.027113,0.007263,0.013543,0.040928,0.009949,0.005618,0.007575,0.178764
sgi,1.053212,0.986123,1.177563,0.977818,1.001297,1.048578,1.132461,0.964083
shi,1.288202,1.217706,1.027145,1.384284,1.769022,1.743103,1.964875,1.453381
shs,1.469785,1.073458,1.990263,1.545701,2.131120,2.202399,2.524659,1.597811


In [47]:
print('Graal VM (Interpreter)')
truffle_interpreter_pic={}
means_shs=[]
means_shi=[]
means_sgi=[]
means_satg=[]
for p in df.program.unique():
    df2 = df.query('vm_version == "8" & vm == "graalvm CE" & program == "%s" & pattern == "truffle + pic"  & idx > 10' % p)
    times = df2.time
    mean = times.mean()
    means_shs.append(hotspot_switch[p]['mean'] / mean)
    means_shi.append(hotspot_interpreter[p]['mean'] / mean)
    means_sgi.append(graal_interpreter[p]['mean'] / mean)
    means_satg.append(truffle_interpreter[p]['mean'] / mean)
    truffle_interpreter_pic[p] = {
        'mean': mean,
        'ci': confidence_interval(times),
        'satg': truffle_interpreter[p]['mean'] / mean,
        'sgi': graal_interpreter[p]['mean'] / mean,
        'shi': hotspot_interpreter[p]['mean'] / mean,
        'shs': hotspot_switch[p]['mean'] / mean,    
    }
    s = ""
    for c in ['vm', 'vm_version', 'vm_full', 'pattern', 'program', 'language',]:
        s += c + ":" + str(df2[c].unique()) + ","
    print(s)
print('mean satg', gmean(means_satg))
print('mean sgi', gmean(means_sgi))
print('mean shi', gmean(means_shi))
print('mean shs', gmean(means_shs))
pd.DataFrame(truffle_interpreter_pic)

Graal VM (Interpreter)
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['b_fibonacci'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['b_sort'],language:['boa'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['f_buffers'],language:['fsm'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['l_fractal'],language:['logo'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['m_binarytree'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['m_fannkuchredux'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:['truffle + pic'],program:['m_fibonacci'],language:['minijava'],
vm:['graalvm CE'],vm_version:['8'],vm_full:['graalvm CE 8'],pattern:[

,b_fibonacci,b_sort,f_buffers,l_fractal,m_binarytree,m_fannkuchredux,m_fibonacci,m_sort
mean,1.500146,1.565794,6.717137,19.217676,3.399943,3.122013,4.922893,9.750549
ci,0.001702,0.002148,0.069131,0.109366,0.021733,0.008270,0.015876,0.013332
satg,0.927305,1.017566,0.586213,0.418794,0.974057,0.969581,0.949249,1.052128
sgi,0.976649,1.003445,0.690303,0.409504,0.975320,1.016681,1.074987,1.014339
shi,1.194556,1.239097,0.602126,0.579730,1.723128,1.690079,1.865155,1.529143
shs,1.362939,1.092315,1.166719,0.647330,2.075832,2.135404,2.396529,1.681102
